In [1]:
# packages
import numpy as np
import pandas as pd
import scipy.stats as sc
import matplotlib.pyplot as plt
from options_wrangler import OptionsWrangler

In [2]:
# data loading
d_loc = 'csv_data/'
ytw = pd.read_csv(d_loc + 'ytw.csv')
ytw['ytw'] = ytw['ytw'].str.rstrip('%').astype('float') / 100.0
ytw['date'] = pd.to_datetime(ytw['date'])
ytm = pd.read_csv(d_loc + 'ytm.csv')
ytm['ytm'] = ytm['ytm'].str.rstrip('%').astype('float') / 100.0;
ytm['date'] = pd.to_datetime(ytm['date'])
expos = pd.read_csv(d_loc + "expos.csv")
expos['date'] = pd.to_datetime(expos['date'])
uso = pd.read_csv(d_loc + 'uso.csv')
uso['trade_date'] = pd.to_datetime(uso['trade_date'])
uso['expirDate'] = pd.to_datetime(uso['expirDate'])
uso_stock = uso[['trade_date', 'stkPx']].groupby('trade_date').agg(np.mean)
monthly_expos = expos[expos.expo_type == 'Monthly']
max_date_back = max([
    min(uso_stock.index), 
    min(monthly_expos['date']), 
    min(ytm['date']), 
    min(ytw['date'])])

uso = uso[uso["trade_date"] >  max_date_back]

In [3]:
uso.head(), max_date_back

(        stkPx trade_date  expirDate  strike  cOi    pOi  cValue  pValue  \
 210074  41.24 2010-04-01 2010-04-16    18.0   50   1040   23.24     0.0   
 210075  41.24 2010-04-01 2010-04-16    19.0   50   2407   22.24     0.0   
 210076  41.24 2010-04-01 2010-04-16    20.0   10   2292   21.24     0.0   
 210077  41.24 2010-04-01 2010-04-16    21.0    0  25400   20.24     0.0   
 210078  41.24 2010-04-01 2010-04-16    22.0    0  21096   19.24     0.0   
 
         delta  gamma  
 210074    1.0    0.0  
 210075    1.0    0.0  
 210076    1.0    0.0  
 210077    1.0    0.0  
 210078    1.0    0.0  , Timestamp('2010-03-31 00:00:00'))

In [4]:
#### USO Sorting
# putting into one frame
# pd.merge
# pd.shift (for advancing)
uso_expos = monthly_expos.merge(uso, left_on='date', right_on='expirDate')
uso_expos = uso_expos.drop(columns=['date', 'expo_type'])
uso_trading = monthly_expos.merge(uso_expos, left_on='date', right_on='trade_date')
uso_trading = uso_trading.sort_values(['trade_date', 'expirDate', 'strike'])

In [5]:
min(np.unique(uso_trading.trade_date)), min(np.unique(uso.trade_date)), min(np.unique(monthly_expos.date))

(numpy.datetime64('2010-12-17T00:00:00.000000000'),
 numpy.datetime64('2010-04-01T00:00:00.000000000'),
 numpy.datetime64('2007-01-20T00:00:00.000000000'))

In [6]:
# building a model on linear regression time series 
# https://towardsdatascience.com/prediction-task-with-multivariate-timeseries-and-var-model-47003f629f9

### Plotting YTM & YTW
This is to visualize the estimated cost of a goverment paying the TEY of the S&P IG Energy Sector YTM & YTW and buying the 20 delta USO put for two months out.

In [7]:
max_date_back

Timestamp('2010-03-31 00:00:00')

In [8]:
delta = .2
put_delta = 1 - delta
first_trade, trade_pass = True, True
for trade_date, chain in uso_trading.groupby('trade_date'):
    first_trade = True
    for expirDate, expo_chain in chain.groupby('expirDate'):
        if first_trade: 
            first_trade = False
            trade_pass = False
            continue
        elif trade_pass:
            break
        else:
            right_delta = expo_chain.iloc[(expo_chain['delta'] - put_delta).abs().argsort()[:1]]
            right_cols = right_delta[['trade_date', 'expirDate', 'pValue', 'strike', 'delta']]
            print(right_cols)
            trade_pass = True

   trade_date  expirDate  pValue  strike   delta
47 2010-12-17 2011-01-21    0.38    35.0  0.8112
    trade_date  expirDate  pValue  strike   delta
283 2011-01-21 2011-02-18     0.3    35.0  0.8204
    trade_date  expirDate  pValue  strike   delta
468 2011-02-18 2011-03-18    0.27    34.0  0.8241
    trade_date  expirDate  pValue  strike   delta
675 2011-03-18 2011-04-15     0.6    38.0  0.7754
     trade_date  expirDate  pValue  strike  delta
1002 2011-04-15 2011-05-20    0.53    41.0  0.777
     trade_date  expirDate  pValue  strike   delta
1282 2011-05-20 2011-06-17    0.44    36.0  0.8193
     trade_date  expirDate  pValue  strike   delta
1567 2011-06-17 2011-07-15    0.49    34.0  0.7822
     trade_date  expirDate  pValue  strike   delta
1857 2011-07-15 2011-08-19    0.41    35.0  0.8153
     trade_date  expirDate  pValue  strike   delta
2121 2011-08-19 2011-09-16    0.59    28.0  0.8164
     trade_date  expirDate  pValue  strike   delta
2410 2011-09-16 2011-10-21    0.54    30.0 

      trade_date  expirDate  pValue  strike   delta
27166 2018-03-16 2018-04-20    0.14    12.0  0.7512
      trade_date  expirDate  pValue  strike   delta
27362 2018-04-20 2018-05-18    0.13    13.0  0.7877
      trade_date  expirDate  pValue  strike   delta
27560 2018-05-18 2018-06-15    0.07    13.5  0.8545
      trade_date  expirDate  pValue  strike   delta
27724 2018-06-15 2018-07-20     0.1    12.0  0.8416
      trade_date  expirDate  pValue  strike   delta
27882 2018-07-20 2018-08-17    0.14    13.5  0.7678
      trade_date  expirDate  pValue  strike   delta
28050 2018-08-17 2018-09-21    0.18    13.0  0.7507
      trade_date  expirDate  pValue  strike   delta
28196 2018-09-21 2018-10-19     0.1    14.0  0.8281
      trade_date  expirDate  pValue  strike   delta
28368 2018-10-19 2018-11-16    0.17    14.0  0.7571
      trade_date  expirDate  pValue  strike   delta
28566 2018-11-16 2018-12-21    0.22    11.0  0.7745
      trade_date  expirDate  pValue  strike   delta
28802 2018-1

In [9]:
wrangler = OptionsWrangler(data=options_data)
# returns the expiration chain for an option, either for a particular month or for a full chain
wrangler.get_expo_chain(expo_chain, trade_date) # throws not valid expo, not in dataset (before, after)
# returns the daily data for an option
wrangler.get_option_history(strike, expo, starting_trade_date) # throws no such option, not in dataset
# gets the histoprical chains for a particular expiration month of options
wrangler.get_days_historical_chain() # throws not in data set

# gets the value of a call at a particular strike
wrangler.get_option_value(expo, trade_date, strike, side) # throws not in dataset 
# gets the market, with sizes for a particular option
wrangler.get_option_market(expo, trade_date, strike, side) # throws not in dataset 

NameError: name 'options_data' is not defined

In [10]:
uso = pd.read_csv(d_loc + 'uso.csv')
uso['trade_date'] = pd.to_datetime(uso['trade_date'])
uso['expirDate'] = pd.to_datetime(uso['expirDate'])
uso_set = uso.set_index(['trade_date', 'expirDate', 'strike']);

In [11]:
uso_wrangler = OptionsWrangler(data=d_loc + 'uso.csv')

In [12]:
trade_date = pd.to_datetime('2020-02-14')
expo = pd.to_datetime('2020-04-17')
strike = 12.0

In [ ]:
%timeit uso.query(f"trade_date == '2020-02-14' and expirDate == '2020-04-17' and strike == 12.0")

In [ ]:
uso.reset_index();

In [ ]:
%timeit uso.query(f"trade_date == '2020-02-14' and expirDate == '2020-04-17' and strike == 12.0")

In [ ]:
    %timeit uso.query("'2020-02-14' < trade_date < '2020-03-15' and expirDate == '2020-04-17' and strike == 12.0 ")

In [ ]:
v = uso.query("'2019-11-20' < trade_date < '2020-03-15' and expirDate == '2020-04-17' and strike == 8")

In [ ]:
%timeit uso_set.query("'2018-11-20' < trade_date < '2020-03-15' and expirDate == '2020-04-17' and strike == 10.0 ")

In [ ]:
%timeit uso.query("'2018-11-20' < trade_date < '2020-03-15' and expirDate == '2020-04-17' and strike == 10.0 ")

In [15]:
lol = uso_wrangler.get_option_history(start_trade_date='2020-02-14', expiration= '2020-04-17', strike=10, side='c')

In [20]:
rmvd = lol.reset_index().drop_duplicates(['trade_date', 'expirDate', 'strike'])

In [22]:
rmvd.set_index(['trade_date', 'expirDate', 'strike'])

,,,stkPx,cValue,cOi
trade_date,expirDate,strike,,,
2020-02-14,2020-04-17,10.0,10.94,1.22,21047
2020-02-18,2020-04-17,10.0,10.94,1.19,20938
2020-02-19,2020-04-17,10.0,11.18,1.36,20968
2020-02-20,2020-04-17,10.0,11.27,1.44,21011
2020-02-21,2020-04-17,10.0,11.16,1.35,10250
2020-02-24,2020-04-17,10.0,10.74,1.05,8905
2020-02-25,2020-04-17,10.0,10.41,0.86,8633
2020-02-26,2020-04-17,10.0,10.19,0.71,8437
2020-02-27,2020-04-17,10.0,9.82,0.53,8605
